# Remove other Files script

In [ ]:
# Remove other Files script

otherFiles <- list.files("/home/zackymo/Desktop/birth_data_zach_calcs_50_V1", pattern="*DateTime.txt", full.names = TRUE)
otherFiles1 <- list.files("/home/zackymo/Desktop/birth_data_zach_calcs_50_V1", pattern="*NMRids.txt", full.names = TRUE)

file.remove(dir(  
  "/home/zackymo/Desktop/birth_data_zach_calcs_50_V1", 
  pattern = "*TimeSeconds.txt", 
  full.names = TRUE
))

# Can also even create a pattern variable that is a vector of some sort and input the patterns in there
# Create a loop that utilizes the pattern as a variable, then go to the next pattern 

# New HammingDistance Script after StackOverflow edits - Different then Dr. Kress's calcs 

In [ ]:
# This version of code works with multiple data files in a particular folder, 
# performs some matrix calculations on them and outputs the new matrices with their corresponding titles 
# into that same folder - currently there are 165 stateFiles within this particular folder which 
# means 330 additional files were created in that same folder in about a minutes worth of time. 

# Some edits are thanks to the critiques from @flodel on stackoverflow
# Final Script after edits 

# Next steps: My calcs are different then Dr. Kress's, need to confirm 
# who is right and who is wrong by running it on some small examples 

# The matrix is actually not thresholded, we would like to save this unthresholded matrix 
# to a file for further verification, also I changed "for (j in i:numRats)"
# and this makes the diagonal of the thresholded matrix to 1, need to
# zero that out

# Establish working directory
workingDir <- "/home/zackymo/Desktop/Sample"
setwd(workingDir)

# Read in State Files from the working directory, and control 
stateFiles <- list.files(pattern = "*state.txt", full.names=FALSE)
thres <- .5

# Next Line for debugging purposes 
# file <- "/home/zackymo/Desktop/Sample/5-14-2014streamW728PPstate.txt"

# This statement automagically reads the data into a dataframe 
df <- read.table(file)

# Create some data frame specific vars
numRats <- ncol(rawData)
numObs <- nrow(rawData)

# Create matrices 
incidence <- diag(numObs, numRats)
thresholded <- diag(0, numRats)

# Fill incidence and thresholded matrices 
for (i in 1:(numRats-1)) {
    for (j in i:numRats) {
      incidence[i,j] <- incidence[j,i] <- sum(df[,i] == df[,j])
      thresholded[i,j] <- thresholded[j,i] <- incidence[i,j]/nrow(df) #>= thres
    }
}

# Adjust the file name strings this way I can save matrices to a new file
outFile <- sub("txt$", "toInc", file)
outFile1 <- sub("txt$", "toThres", file)

#Creating the various output files 
write.table(incidence, file = outFile, 
            row.names = FALSE, 
            col.names = FALSE)

write.table(thresholded, file = outFile1, 
            row.names = FALSE, 
            col.names = FALSE)

# }


# Old Script before StackOverflow edits 

In [ ]:
# This version of code works with multiple data files in a particular folder, 
# performs some matrix calculations on them and outputs the new matrices with their corresponding titles 
# into that same folder - currently there are 165 stateFiles within this particular folder which 
# means 330 additional files were created in that same folder in about a minutes worth of time. 
# 6.875 days are in with all of these files  

stateFiles <- list.files("/home/zackymo/Desktop/birth_data_zach_calcs_75/", pattern="*state.txt", full.names=TRUE)
thres <- .5

for(file in stateFiles){
  rawData = read.table(file)
  df <- data.frame(rawData)
  incidence <- matrix(rep(0, ncol(df)*ncol(df)), nrow=ncol(df))
  thresholded <- matrix(rep(0, ncol(df)*ncol(df)), nrow=ncol(df)) 
  
  #Set the diagonal = to constants due to the fact that both are symmetric matrices 
  diag(incidence) <- nrow(df)
  diag(thresholded) <- 0
  
  #Describe for loop here 
  for (i in 1:(ncol(df)-1)) {
    for (j in (i+1):ncol(df)) {
      incidence[i,j] = incidence[j,i] = sum(df[,i] == df[,j])
      if(incidence[i,j]/nrow(df) >= thres){
        thresholded[i,j] = thresholded[j,i] = 1
      }#should these conditions be reversed for effiency? How can I check that? 
      else{
        thresholded[i,j] = thresholded[j,i] = 0
      }
    }
  }
  #Data preparation for output file - transposing matrices 
  tincidence <- t(incidence) 
  tthresholded <- t(thresholded)
  
  #Manipulating the fileName to be able to create a new file w/ a meaningful file name
  #I may be able to split by / then edit the paste below f[1] + f[2] + f[3] + f[4] + f[5]
  f <- strsplit(file, split='.', fixed=TRUE)
  f <- unlist(f)
  
  #This will not work -> because you end up 
  #Setting the current directory for the output files to be placed, and never 
  #setwd("/home/zackymo/Desktop/birth_data1_out_files")
  
  #Creating the various output files 
  write(tincidence, file = paste(f[1],"toInc", sep = ""),
        ncolumns = ncol(df),
        sep = " ") 
  
  write(tthresholded, file = paste(f[1],"toThresholded", sep = ""),
        ncolumns = ncol(df),
        sep = " ") 
}
